# Домашнее задание № 3. Исправление опечаток

In [98]:
!pip install textdistance
!pip install razdel
import textdistance
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [99]:
from google.colab import files
uploaded = files.upload()

Saving wiki_data.txt to wiki_data (1).txt


In [29]:
from google.colab import files
uploaded = files.upload()

Saving sents_with_mistakes.txt to sents_with_mistakes (1).txt


In [30]:
from google.colab import files
uploaded = files.upload()

Saving correct_sents.txt to correct_sents.txt


## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [4]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [100]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))

In [106]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):

    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=8, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    # Если закомментировать строку ниже, то получим улучшенный алгоритм 
    # return closest
    
    # generate list with same match
    first_closest = closest[0]
    same_closest = []
    for c in closest:
        if c[1] == first_closest[1]:
            same_closest.append(c)
    # get most frequent from list
    max = 0
    res = []
    for c in same_closest:
        probability = P(c[0])
        if probability > max:
            max = probability
            res = c
    return [ c ]

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [107]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

In [108]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [109]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

mistakes = []
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 50:
        print(i)


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900


In [105]:
# исходный алгоритм

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8535267633816909
0.4720496894409938
0.09004249454461927


In [110]:
# улучшенный

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8538269134567283
0.47437888198757766
0.09004249454461927


Улучшенный алгоритм работает незначительно эффективнее оригинального, процент правильных слов и исправленных ошибок немного повысился.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [111]:
deletions_dict = {}

def gen_deletions(word):
    i = 0
    result = []
    while i < len(word):
        reduced_word = word[:i] + word[i+1:]
        result.append(reduced_word)
        i += 1
    return result

def gen_deletions_dict(voc):
    for key in voc:
        #print('key: {}'.format(key))
        for d in gen_deletions(key):
            #print('d: {}'.format(d))
            if d in deletions_dict:
                deletions_dict[d].append(key)
            else:
                deletions_dict[d] = [ key ]

def symspell(word):
    #step 3:
    #generate deletions of misspelled word and find appropriate word from deletions_dict
    #print('symspell input: {}'.format(word))
    words = gen_deletions(word)
    #print('symspell generated deletions: {}'.format(words))
    founded_words = []
    for w in words:
        if w in deletions_dict:
            founded_words = deletions_dict[w]
            break
    #print('symspell words list: {}'.format(founded_words))
    #step 4
    #search most frequent word
    max = 0
    result_word = ''
    for w in founded_words:
        if vocab[w] > max:
            max = vocab[w]
            result_word = w
    return result_word

gen_deletions_dict (vocab)
#print (symspell('блом'))


In [112]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        #predicted = cashed.get(pair[1], correction(pair[1]))
        predicted = cashed.get(pair[1], symspell(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [113]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6035017508754377
0.17468944099378883
0.33306534971861723


Если сравнивать с алгоритмом Норвига, то процент правильных слов и исправленных ошибок упал, при этом процент ошибочно исправленных правильных слов. Это объясняется тем, что мы используем только одну операцию удаления.

## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [114]:
def method_eval(X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec, topn=topn, metric=metric))
                cashed[pair[1]] = pred
            else:
                pred = pair[1]


            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)
    
    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)

In [115]:
# default parameters
%%time
vec = CountVectorizer(analyzer = 'char', ngram_range = (1,3), max_features = 1000)
X = vec.fit_transform(vocab)

method_eval(X, vec)

0
100
200
300
400
500
600
700
800
900
0.7926963481740871
0.0
0.09004249454461927
CPU times: user 5min 55s, sys: 2.59 s, total: 5min 58s
Wall time: 5min 56s


In [116]:
# слишком маленький topn в get_closest_match_vec приводит к тому,
# что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить

%%time
vec = CountVectorizer(analyzer = 'char', ngram_range = (1,3), max_features = 1000)
X = vec.fit_transform(vocab)

method_eval(X, vec, topn = 30)

0
100
200
300
400
500
600
700
800
900
0.7926963481740871
0.0
0.09004249454461927
CPU times: user 6min 31s, sys: 2.13 s, total: 6min 33s
Wall time: 6min 32s
